<a href="https://colab.research.google.com/github/leegonzales/LangChainExamples/blob/main/Langchain_transcription_with_sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Query the YouTube video transcripts, returning timestamps as sources to legitimize the answers by [@m_morzywolek](https://twitter.com/m_morzywolek)

In [ ]:
# First set runtime to GPU

In [ ]:
pip install pytube # For audio downloading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 1.5 MB/s 


In [ ]:
pip install git+https://github.com/openai/whisper.git -q # Whisper from OpenAI transcription model

     |████████████████████████████████| 5.8 MB 5.5 MB/s 
     |████████████████████████████████| 7.6 MB 69.9 MB/s 
     |████████████████████████████████| 182 kB 51.7 MB/s 


In [ ]:
import whisper 
import pytube 

In [ ]:
url = "https://www.youtube.com/watch?v=UF8uR6Z6KLc&ab_channel=Stanford"
video = pytube.YouTube(url)

In [ ]:
audio = video.streams.get_audio_only()
audio.download(filename='tmp.mp3') # Downlods only audio from youtube video

'/content/tmp.mp3'

In [ ]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 57.1MiB/s]


In [ ]:
transcription = model.transcribe('/content/tmp.mp3')

In [ ]:
res = transcription['segments']

In [ ]:
from datetime import datetime

def store_segments(segments):
  texts = []
  start_times = []

  for segment in segments:
    text = segment['text']
    start = segment['start']

    # Convert the starting time to a datetime object
    start_datetime = datetime.fromtimestamp(start)

    # Format the starting time as a string in the format "00:00:00"
    formatted_start_time = start_datetime.strftime('%H:%M:%S')

    texts.append("".join(text))
    start_times.append(formatted_start_time)

  return texts, start_times

In [ ]:
store_segments(res)

([' This program is brought to you by Stanford University.',
  ' Please visit us at stanford.edu.',
  ' Thank you.',
  " I'm honored to be with you today for your commencement from one of the finest universities",
  ' in the world.',
  " Truth be told, I never graduated from college and this is the closest I've ever gotten",
  ' to a college graduation.',
  ' Today I want to tell you three stories from my life.',
  " That's it.",
  ' No big deal.',
  ' Just three stories.',
  ' The first story is about connecting the dots.',
  ' I dropped out of Reed College after the first six months but then stayed around as a drop-in',
  ' for another 18 months or so before I really quit.',
  " So why'd I drop out?",
  ' It started before I was born.',
  ' My biological mother was a young unwed graduate student and she decided to put me up for adoption.',
  ' She felt very strongly that I should be adopted by college graduates so everything was all',
  ' set for me to be adopted at birth by a lawyer

In [ ]:
texts, start_times = store_segments(res)

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [ ]:
pip install --upgrade faiss-gpu==1.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89.7 MB 35 kB/s 


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chains import VectorDBQAWithSourcesChain
from langchain import OpenAI
import openai
import faiss

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs = []
metadatas = []
for i, d in enumerate(texts):
    splits = text_splitter.split_text(d)
    docs.extend(splits)
    metadatas.extend([{"source": start_times[i]}] * len(splits))
embeddings = OpenAIEmbeddings()

In [ ]:
store = FAISS.from_texts(docs, embeddings, metadatas=metadatas)
faiss.write_index(store.index, "docs.index")

In [ ]:
chain = VectorDBQAWithSourcesChain.from_llm(llm=OpenAI(temperature=0), vectorstore=store)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
result = chain({"question": "How old was Steve Jobs when started Apple?"})

In [ ]:
print(f"Answer: {result['answer']}  Sources: {result['sources']}")

Answer:  Steve Jobs was 20 when he started Apple.  Sources: 00:05:47, 00:05:59
